# 1. 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from keras import backend as K
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, LSTM, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 불러오기

In [2]:
final_train_path = os.listdir('./data/final_labeling/train')
final_test_path = os.listdir('./data/final_labeling/test')

In [3]:
final_train_path[18], final_test_path[18]

('서울_day_train.csv', '서울_day_test.csv')

서울 데이터를 활용하여 자연재난을 얼마나 정확하게 예측하는지 확인해보겠습니다.

# 3. 데이터 정규화

In [4]:
# 데이터
col = ['평균_기온', '최저_기온', '최고_기온', '일강수량', '최대_풍속', '최대_풍속_풍향_x', '최대_풍속_풍향_y',
       '평균_풍속', '풍정합', '최다_풍향_x', '최다_풍향_y', '평균_이슬점온도', '최소_상대습도', '평균_상대습도',
       '평균_증기압', '평균_현지기압', '최고_해면_기압', '최저_해면기압', '평균_해면기압', '합계_일조_시간',
       '일_최심신적설', '합계_3시간_신적설', '안개_계속_시간', '체감온도', '열대야']

In [5]:
# train test split
def data_split(final_train_path, final_test_path, weather, label, col):
    df_train = pd.read_csv('./data/final_labeling/train/' + final_train_path)
    df_test = pd.read_csv('./data/final_labeling/test/' + final_test_path)
    
    df_weather = df_train[df_train['계절'] == weather]
    df_weather.reset_index(drop=True, inplace=True)
    y_train = df_weather[label]
    x_train = df_weather.loc[:, col].copy()
    
    x_test = df_test[df_test['계절'] == weather]
    x_test.reset_index(drop=True, inplace=True)
    y_test = x_test[label]
    x_test = x_test.loc[:, col].copy()

    # 수치형 데이터 정규화
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)

    x_train = pd.DataFrame(x_train)
    x_train.columns = col

    x_test = pd.DataFrame(x_test)
    x_test.columns = col

    return x_train, x_test, y_train, y_test

In [6]:
def make_dataset(data, label, window_size=30):
    feature_list = []
    label_list = []
    
    # 3차원으로 변형 → np.reshape(samples, time steps, features)
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))

    return np.array(feature_list), np.array(label_list)

# 4. 딥러닝 모델 설계

In [7]:
# StratifiedKFold : 데이터 불균형 해결
skf = StratifiedKFold(n_splits=5, shuffle=True)

# optimizer
adam = Adam(lr=1e-4, decay=1e-7)

In [8]:
def skfold(x_train, x_test, y_train, y_test, n):
    # StratifiedKFold 객체는 split에 라벨도 함께 적용하는 것으로, 라벨이 같은 비율을 갖도록 index를 반환해줌
    for fold, (train_index, test_index) in enumerate(skf.split(np.zeros(len(y)), y)):

        # 하나의 fold를 실행하는 데 많은 시간이 걸리므로 fold 한 개마다 저장하는 식으로 했음
        if fold!=n:
            print(f'skip {fold}')
            continue

        x_feature, y_label = make_dataset(x_train, y_train)
        x_feature_test, y_label_test = make_dataset(x_test, y_test)

        x_tr, x_val, y_tr, y_val = train_test_split(x_feature, y_label, test_size=0.2, shuffle=True)

        # 모델 구성
        model = Sequential()
        model.add(LSTM(256, activation='relu', return_sequences=True, input_shape=x_tr.shape[1:]))
        model.add(Dropout(0.25))
        model.add(LSTM(128, activation='relu', return_sequences=True))
        model.add(Dropout(0.25))
        model.add(LSTM(64, activation='relu', return_sequences=True))
        model.add(Dropout(0.25))
        model.add(LSTM(32, activation='relu', return_sequences=True))
        model.add(Dropout(0.25))
        model.add(LSTM(16, activation='relu', return_sequences=True))
        model.add(Dropout(0.25))
        model.add(LSTM(8, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(1, activation='sigmoid'))

        # compile
        model.compile(optimizer=adam, loss='mse', metrics='acc')

        # 조기종료
        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        # 가중치
        weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)

        # train 학습
        model.fit(x_tr, y_tr, epochs=100, batch_size=64, class_weight={i:weights[i] for i in range(len(weights))}, validation_data=(x_val, y_val), callbacks=[early_stop])
        model.save('./data/model/' + str(fold) + '.keras')

        # test 예측
        y_pred = model.predict(x_feature_test)
        y_pred = np.array(y_pred)
        np.save('./data/model/' + str(fold) + 'keras.npy', y_pred)
        
    return y_label_test, y_pred

시계열 데이터이기 때문에 lstm 모델을 사용했습니다.

데이터 불균형을 해결하기 위해서 StratifiedKFold를 사용하여 라벨의 비율을 고려하여 fold를 나누었습니다.

또한 학습 시 적용할 class_weight를 설정하였습니다.

최종적으로 fold별 예측값을 soft voting하여 모델의 성능을 높이고자 했습니다.

In [9]:
# train, test, validation
x_train, x_test, y_train, y_test = data_split(final_train_path[18], final_test_path[18], '여름철', '여름_폭염', col)

y = pd.concat([y_train, y_test])
y.reset_index(drop=True, inplace=True)

for i in range(5):
    y_label_test, y_pred = skfold(x_train, x_test, y_train, y_test, i)

Epoch 1/100
19/19 [==============================] - 8s 163ms/step - loss: 0.2499 - acc: 0.2767 - val_loss: 0.2501 - val_acc: 0.5400
Epoch 2/100
19/19 [==============================] - 2s 131ms/step - loss: 0.2491 - acc: 0.5158 - val_loss: 0.2490 - val_acc: 0.6300
Epoch 3/100
19/19 [==============================] - 2s 132ms/step - loss: 0.2451 - acc: 0.6100 - val_loss: 0.2315 - val_acc: 0.7667
Epoch 4/100
19/19 [==============================] - 3s 137ms/step - loss: 0.2188 - acc: 0.6825 - val_loss: 0.1705 - val_acc: 0.8000
Epoch 5/100
19/19 [==============================] - 2s 125ms/step - loss: 0.2086 - acc: 0.7258 - val_loss: 0.1587 - val_acc: 0.8333
Epoch 6/100
19/19 [==============================] - 2s 122ms/step - loss: 0.1837 - acc: 0.7508 - val_loss: 0.1651 - val_acc: 0.8067
Epoch 7/100
19/19 [==============================] - 2s 122ms/step - loss: 0.1807 - acc: 0.7558 - val_loss: 0.1557 - val_acc: 0.8200
Epoch 8/100
19/19 [==============================] - 2s 126ms/step - 

19/19 [==============================] - 2s 132ms/step - loss: 0.1453 - acc: 0.9058 - val_loss: 0.0766 - val_acc: 0.8600
Epoch 26/100
19/19 [==============================] - 2s 129ms/step - loss: 0.1479 - acc: 0.8858 - val_loss: 0.0768 - val_acc: 0.8567
Epoch 27/100
19/19 [==============================] - 2s 130ms/step - loss: 0.1464 - acc: 0.9017 - val_loss: 0.0643 - val_acc: 0.8667
Epoch 28/100
19/19 [==============================] - 2s 128ms/step - loss: 0.1464 - acc: 0.9008 - val_loss: 0.0757 - val_acc: 0.8633
Epoch 29/100
19/19 [==============================] - 2s 129ms/step - loss: 0.1480 - acc: 0.8892 - val_loss: 0.0740 - val_acc: 0.8567
Epoch 30/100
19/19 [==============================] - 2s 129ms/step - loss: 0.1448 - acc: 0.8900 - val_loss: 0.0681 - val_acc: 0.8600
Epoch 31/100
19/19 [==============================] - 2s 130ms/step - loss: 0.1444 - acc: 0.8883 - val_loss: 0.0866 - val_acc: 0.8267
Epoch 32/100
19/19 [==============================] - 2s 131ms/step - loss:

19/19 [==============================] - 3s 145ms/step - loss: 0.1991 - acc: 0.7367 - val_loss: 0.2604 - val_acc: 0.7233
Epoch 3/100
19/19 [==============================] - 3s 138ms/step - loss: 0.2032 - acc: 0.7625 - val_loss: 0.1727 - val_acc: 0.8233
Epoch 4/100
19/19 [==============================] - 3s 140ms/step - loss: 0.1789 - acc: 0.8017 - val_loss: 0.2304 - val_acc: 0.7700
Epoch 5/100
19/19 [==============================] - 3s 151ms/step - loss: 0.1733 - acc: 0.7958 - val_loss: 0.2117 - val_acc: 0.7700
Epoch 6/100
19/19 [==============================] - 3s 143ms/step - loss: 0.1640 - acc: 0.7600 - val_loss: 0.1709 - val_acc: 0.7767
Epoch 7/100
19/19 [==============================] - 3s 145ms/step - loss: 0.1557 - acc: 0.7867 - val_loss: 0.1710 - val_acc: 0.7767
Epoch 8/100
19/19 [==============================] - 3s 144ms/step - loss: 0.1509 - acc: 0.7842 - val_loss: 0.1678 - val_acc: 0.7767
Epoch 9/100
19/19 [==============================] - 3s 146ms/step - loss: 0.1589

In [10]:
files = glob.glob('C:/Users/user/ML_220823/pro/data/model/*keras.npy')

predict = np.load(files[0]) + np.load(files[1]) + np.load(files[2]) + np.load(files[3]) + np.load(files[4])
y_pred = np.argmax(predict, axis=1)

# 5. 폭염 예측

In [11]:
# 실제 폭염 라벨링
y_label_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 0., 0., 0., 0.

In [12]:
# 폭염 예측값
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

# 6. 머신러닝과 딥러닝 결과 차이

머신러닝(soft voting) 모델은 유의미한 결과를 얻었습니다.

반면에 딥러닝(LSTM) 모델은 원하는 결과가 나오지 않아서 다양한 방법으로 모델을 수정했지만 결국 유의미한 결과를 얻지 못했습니다.

이러한 문제가 발생한 원인이 무엇인지 고민해 보았습니다.

저희가 생각한 문제점은 다음과 같습니다.

#### 1) 머신러닝 모델에서 사용한 LogisticRegression, RandomForestClassfier, SVC, XGBClassifier의 구조와 딥러닝 모델 구조의 차이

저희는 직접 layer를 쌓아서 모델을 설계하는 방식으로 진행했습니다.

이 부분에서 LSTM 모델의 성능을 충분히 끌어내지 못했다고 생각합니다.

이미지 분류 모델 설계를 예로 들어보겠습니다.

convolution - convolution - maxpooling - dropout의 기초적인 형태로 직접 layer를 쌓아서 만든 모델과 전이학습 모델은 성능의 차이가 상당합니다.

비슷한 이유로, 저희가 LSTM이 충분한 성능을 낼 수 있도록 모델을 설계하지 못했기 때문에 머신러닝 모델에 비해 미흡한 결과를 얻은 것 같습니다.


#### 2) 부적합한 데이터 선택

LSTM 모델은 지표 간 순서와 연관성이 있을 것이라는 가정 하에 사용하는 신경망입니다.

본 프로젝트에서는 일별 데이터를 분석 및 시각화, 예측 모델 설계에 사용했으며, 시간별 데이터는 일별 데이터의 결측치 처리에만 활용했습니다.

하지만 지표 간 순서와 연관성이 중요한 LSTM의 특성을 고려한다면 일별 데이터보다 시간별 데이터를 사용하는 것이 적절합니다.

일별 데이터는 시간별 데이터로부터 24시간 간격으로 최소값, 최대값, 평균값 등을 추출한 데이터이기 때문에 연속성 측면에서는 다소 부족합니다.

이러한 이유로 LSTM 모델로부터 원하는 결과를 얻지 못한 것이라고 생각합니다.